In [0]:
%pip install aim
%pip install langchain
%pip install openai
%pip install google-search-results

In [0]:
import os
from datetime import datetime

import openai
from langchain.llms import AzureOpenAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

In [0]:
openai.api_type = "azure" 
openai.api_base =  "https://kf-llm.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_KEY"] = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["SERPAPI_API_KEY"] = "4ea6865054c8eb8608723170dd33e12314d8e182c10c16ee15565ccf03526e8e"

In [0]:
llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
)

In [0]:
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1: OpenAI LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
openai.temperature = 1.0
openai.callbacks = callbacks

In [0]:
# scenario 1 - LLM
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
aim_callback.flush_tracker(
    langchain_asset=llm,
    experiment_name="scenario 2: Chain with multiple SubChains on multiple generations",
)

In [0]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [0]:
# scenario 2 - Chain
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=callbacks)

test_prompts = [
    {
        "title": "documentary about good video games that push the boundary of game design"
    },
    {"title": "the phenomenon behind the remarkable speed of cheetahs"},
    {"title": "the best in class mlops tooling"},
]
synopsis_chain.apply(test_prompts)
aim_callback.flush_tracker(
    langchain_asset=synopsis_chain, experiment_name="scenario 3: Agent with Tools"
)

In [0]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

In [0]:
%pip install numexpr

In [0]:
# scenario 3 - Agent with Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=callbacks,
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)
aim_callback.flush_tracker(langchain_asset=agent, reset=False, finish=True)

In [0]:
!pip show aim


In [0]:
%sh
aim up

In [0]:
%sh
# ifconfig
curl ifconfig.me